In [1]:
    !which python; python -V;

/home/thomson/devel/perylune/venv/bin/python
Python 3.8.5


In [2]:
from poliastro.bodies import Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, Sun, Pluto
from poliastro.twobody import Orbit

from poliastro.constants import GM_earth

from astropy import units as u
import numpy as np

from perylune.orbit_tools import *
from perylune.interplanetary import *
from perylune.constants import *

In [11]:
# Let's create two departure orbits. Typically the departure orbit is circular (orb1), but we can make some
# interesting observations on eliptical orbit (orb2) and study how the escape velocity changes between apoapsis and periapsis.
orb1 = Orbit.circular(Earth, 250*u.km)
orb2 = Orbit.from_classical(Earth, 350*u.km + Earth.R, 0.14285714285714285*u.one, 0*u.deg, 0*u.deg, 0*u.deg, 0*u.deg)

In [12]:
print_orb(orb1)
print_orb(orb2)

6628 x 6628 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)
a(𝑎)=6628.14km, b=6628.14km, e=0.00, i=0.00deg raan(Ω)=0.00deg argp(𝜔)=0.00deg nu(𝜈)=0.00deg
period=5370.30s perapis=6628km(250.00km) apoapsis=6628km(250.00km)
5767 x 7689 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)
a(𝑎)=6728.14km, b=6659.13km, e=0.14, i=0.00deg raan(Ω)=0.00deg argp(𝜔)=0.00deg nu(𝜈)=0.00deg
period=5492.29s perapis=5767km(-611.16km) apoapsis=7689km(1311.16km)


In [13]:
# Calculate delta-v necessary to reach escape velocity for circular orbit. The values returned are current position, periapsis, apoapis.
# No surprises here - it's perfectly circular, so it's always the same.
escape_delta_v(orb1, False)

(<Quantity 3212.16227604 m / s>,
 <Quantity 3212.16227604 m / s>,
 <Quantity 3212.16227604 m / s>)

In [6]:
# The situation for eliptical orbit is more interesting. The delta-v for periapsis burn is 2869 m/s, which is the smallest value
# for this orbit and up to 3516 for apoapsis (when the velocity is smallest, so extra delta-v is needed to escape)
escape_delta_v(orb2, False)

(<Quantity 2869.63167184 m / s>,
 <Quantity 2869.63167184 m / s>,
 <Quantity 3516.37639716 m / s>)

In [7]:
# Here's an utility function that returns a (semi-major axis) and e (eccentricity) for min and max altitude orbits
apses_to_ae(400, 300)
apses_to_ae(300*u.km , 400*u.km, Earth.R)

(350.0, 0.14285714285714285)

In [22]:
# To obtain data, head to Horizons (https://ssd.jpl.nasa.gov) and use the following parameters:
# Ephemepris type: ELEMENTS
# Target body: <whatever planet needed, e.g. Mars>
# Center: Sun
# Time span: pick any date
# Table settings: output inits=KM-S
# Display/Output: plain text

# For example, Earth returns the following data for 2020-Nov-03
#2459156.500000000 = A.D. 2020-Nov-03 00:00:00.0000 TDB 
# EC= 1.623243125148644E-02 QR= 1.470544578776156E+08 IN= 4.337517056126222E-03
# OM= 1.956969904857147E+02 W = 2.693660952667993E+02 Tp=  2459219.993764857296
# N = 1.142090613016157E-05 MA= 2.973464932363198E+02 TA= 2.956789660740816E+02
# A = 1.494808962493945E+08 AD= 1.519073346211734E+08 PR= 3.152114165874045E+07

a = 1.494808962493945E+08 * u.km
e = 1.623243125148644E-02 * u.one
inc = 4.337517056126222E-03 * u.deg
raan = 1.956969904857147E+02 * u.deg
argp = 2.693660952667993E+02 * u.deg
nu = 2.956789660740816E+02 * u.deg

# Orbital data from Horizons
orbit_earth = Orbit.from_classical(Sun, a, e, inc, raan, argp, nu)

# MARS
# EC= 9.333873836923647E-02 QR= 2.066557040253017E+08 IN= 1.847950769090515E+00
# OM= 4.949400395133841E+01 W = 2.866239949629700E+02 Tp=  2459064.901374706533
# N = 6.065617422097314E-06 MA= 4.800403158507560E+01 TA= 5.659109019293314E+01
# A = 2.279304441149290E+08 AD= 2.492051842045562E+08 PR= 5.935092422553787E+07
a = 2.279304441149290E+08 * u.km
e = 1.623243125148644E-02 * u.one
inc = 1.847950769090515E+00 * u.deg
raan = 4.949400395133841E+01 * u.deg
argp = 2.866239949629700E+02  * u.deg
nu = 5.659109019293314E+01 * u.deg

# TODO: insert actual date
orbit_mars = Orbit.from_classical(Sun, a, e, inc, raan, argp, nu)

hoh = Maneuver.hohmann(orbit_earth, orbit_mars.a)
orb3,orb4 = orbit_earth.apply_maneuver(hoh, intermediate=True)

In [24]:
from poliastro.plotting import OrbitPlotter2D

op = OrbitPlotter2D()
op.plot(orbit_earth, label="Earth")
op.plot(orbit_mars, label="Mars")
#op.plot(orb3, label="transfer")
#op.plot(orb4, label="arrival")
#op.plot(orb3, label="Transfer orbit - Hohmann transfer")
#op.plot(orb4, label="Final orbit - GEO")

Number of impulses: 2, Total cost: 5.598702 km / s

In [21]:
get_cost(hoh, 0)

<Quantity 2838.71961742 m / s>